## Tiny Blocks Examples

In [1]:
# extract blocks
from tiny_blocks.extract import FromCSV
from tiny_blocks.extract import FromSQLQuery

# transform blocks 
from tiny_blocks.transform import DropDuplicates
from tiny_blocks.transform import Fillna
from tiny_blocks.transform import Rename
from tiny_blocks.transform import Merge

# load blocks
from tiny_blocks.load import ToCSV
from tiny_blocks.load import ToSQL

# pipeline operations
from tiny_blocks import FanIn, FanOut, Tee

# mock data
from tests.conftest import add_mocked_data
from tests.conftest import delete_mocked_data

import pandas as pd

ImportError: cannot import name 'add_mocked_data' from 'tests.conftest' (/code/tests/conftest.py)

In [ ]:
add_mocked_data()

### Example 1. Basic Pipeline 

In [ ]:
# extract blocks
from_sql = FromSQL(path=)

# transform blocks
fillna = Fillna(value="Hola Mundo")
drop_duplicates = DropDuplicates()

# load blocks
to_csv = ToCSV(path)

In [ ]:
# pipeline

from_sql >> fillna >> drop_duplicates >> to_csv

In [ ]:
# check if results exist

df = pd.read_csv(to_csv.path)
df

### Example 2. Merging Pipes

In [ ]:
# create blocks
merge = Merge()
to_csv = ToCSV(path)

In [ ]:
# pipeline

FanIn(from_sql >> fillna, from_csv) >> merge >> drop_duplicates >> to_csv

In [ ]:
# check if results exist

df = pd.read_csv(to_csv.path)
df

### Example 3. FanOut

In [ ]:
# pipeline

from_sql >> fillna >> FanOut(to_sql) >> rename >> to_csv

In [ ]:
# check if results exist

df = pd.read_csv(to_csv.path)
df

### Example 4. Branching with Tee

In [ ]:
# pipeline

pipe_1 = drop_dupl >> to_csv
pipe_2 = rename >> to_sql

from_sql >> fillna >> Tee(pipe_1, pipe_2)

# # same as
# from_sql >> fillna >> Tee(drop_dupl >> to_csv, rename >> to_sql)

In [ ]:
# check if results exist

df = pd.read_csv(to_csv.path)
df

In [ ]:
# tear down dbs and files

In [ ]:
delete_mocked_data()